In [16]:
import json
from json import JSONDecodeError
import pandas as pd
import os
from dotenv import load_dotenv #pip install python-dotenv

import requests
import datetime
from src.db_datos import Db_datos


In [17]:
def convert_unix_to_date(unix_time):
    return datetime.datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

In [26]:
def get_dataWeather(**kwargs):

    url = 'http://api.openweathermap.org/'

    key = 'f2ab75730071c8df43c0929be2a42793'
    lat = kwargs.get('lat', '')
    lon =  kwargs.get('lon', '')
    start= kwargs.get('start', '')
    end = kwargs.get('end', '')

    endpoint = str('data/2.5/air_pollution/history?lat={}&lon={}&start={}&end={}&appid={}').format(lat, lon, start, end, key)

    headers = {
            "Content-Type": "application/json"
        }
    response = requests.get(url + endpoint, headers=headers)
    if response.status_code == 200:
        try:
            # Obtener el token de la respuesta
            data = response.json()
            print('Process ok')
            return data
        except json.decoder.JSONDecodeError as e:
            print("Error de decodificación JSON:", str(e))
            return 'Error'  


In [31]:
lat= '4.710989'
lon= '-74.072092'
start ='1577854800'
end = '1719810000'
data = get_dataWeather(lat= lat,
                        lon= lon,
                        start = start,
                        end = end)


In [41]:
df = pd.json_normalize(data['list'])
df['date'] = df['dt'].apply(convert_unix_to_date)
df['lat'] = lat
df['lon'] = lon
df = df.drop('dt',axis=1)


titulos = ['main.aqi',
            'components.co',
            'components.no',
            'components.no2',
            'components.o3',
            'components.so2',
            'components.pm2_5',
            'components.pm10',
            'components.nh3',
            'date',
            'lat',
            'lon']


df.columns = titulos


C:\Users\c2222\AppData\Local\Temp\ipykernel_27360\2429585078.py:2: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')


In [44]:
dc_toPush = [{'column_sql': 'co', 'column_df': 'components.co'},
            {'column_sql': 'no', 'column_df': 'components.no'},
            {'column_sql': 'no2', 'column_df': 'components.no2'},
            {'column_sql': 'o3', 'column_df': 'components.o3'},
            {'column_sql': 'so2', 'column_df': 'components.so2'},
            {'column_sql': 'pm2_5', 'column_df': 'components.pm2_5'},
            {'column_sql': 'pm10', 'column_df': 'components.pm10'},
            {'column_sql': 'nh3', 'column_df': 'components.nh3'},
            {'column_sql': 'lon', 'column_df': 'lon'},
            {'column_sql': 'lat', 'column_df': 'lat'},
            {'column_sql': 'aqi', 'column_df': 'main.aqi'},
            {'column_sql': 'date', 'column_df': 'date'}]

con = Db_datos()
con.pushData(df,'pollution',False,dc_toPush)

31229